this scripts will filter and regress out flagged timepoints and motion regressors; a side-by-side comparison of this to SPM denoised data reveals that they are both doing fairly similar things which gives me great confidence that I did this correctly!

## load packages

In [1]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 

import numpy as np
from nipype.interfaces import afni
import nibabel as nib
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
from pandas import *

%autosave 5
%matplotlib inline

/Users/alexbarnett/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Autosaving every 5 seconds


## set directories


In [2]:
data_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/'
analysis_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/'
subjlist = pd.read_excel(analysis_dir+'subjlist.xlsx')


## loop through subjects and denoise SRM movie

In [3]:
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    tproject.inputs.bandpass = (0.001, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run()  
 

211116-11:18:00,829 nipype.interface INFO:
	 stderr 2021-11-16T11:18:00.828965:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:18:00,830 nipype.interface INFO:
	 stderr 2021-11-16T11:18:00.828965:++ Authored by: Cox the Algebraic (Linear)
211116-11:18:00,925 nipype.interface INFO:
	 stderr 2021-11-16T11:18:00.925230:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-000/ses-02/func/sub-000_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:18:00,925 nipype.interface INFO:
	 stderr 2021-11-16T11:18:00.925230:     Warnings of this type will be muted for this session.
211116-11:18:00,926 nipype.interface INFO:
	 stderr 2021-11-16T11:18:00.925230:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:18:06,710 nipype.interface INFO:
	 stderr 2021-11-16T11:18:06.709996:++ Setting up regressors
211116-11:18:06,710 nipype.i

211116-11:19:12,310 nipype.interface INFO:
	 stderr 2021-11-16T11:19:12.310653:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:19:12,312 nipype.interface INFO:
	 stderr 2021-11-16T11:19:12.310653:++ Authored by: Cox the Algebraic (Linear)
211116-11:19:12,321 nipype.interface INFO:
	 stderr 2021-11-16T11:19:12.320975:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-103/ses-01/func/sub-103_ses-01_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:19:12,325 nipype.interface INFO:
	 stderr 2021-11-16T11:19:12.320975:     Warnings of this type will be muted for this session.
211116-11:19:12,325 nipype.interface INFO:
	 stderr 2021-11-16T11:19:12.320975:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:19:15,788 nipype.interface INFO:
	 stderr 2021-11-16T11:19:15.788073:++ Setting up regressors
211116-11:19:15,788 nipype.i

211116-11:20:22,550 nipype.interface INFO:
	 stderr 2021-11-16T11:20:22.547861:++ Authored by: Cox the Algebraic (Linear)
211116-11:20:22,550 nipype.interface INFO:
	 stderr 2021-11-16T11:20:22.550916:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-106/ses-02/func/sub-106_ses-02_task-srmmovie_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:20:22,551 nipype.interface INFO:
	 stderr 2021-11-16T11:20:22.550916:     Warnings of this type will be muted for this session.
211116-11:20:22,551 nipype.interface INFO:
	 stderr 2021-11-16T11:20:22.550916:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:20:24,569 nipype.interface INFO:
	 stderr 2021-11-16T11:20:24.569233:++ Setting up regressors
211116-11:20:24,569 nipype.interface INFO:
	 stderr 2021-11-16T11:20:24.569233:++ setting up stopband frequency mask
211116-11:20:24,570 nipype.interface INFO:
	 stderr 20

211116-11:21:28,77 nipype.interface INFO:
	 stderr 2021-11-16T11:21:28.076641:     Warnings of this type will be muted for this session.
211116-11:21:28,78 nipype.interface INFO:
	 stderr 2021-11-16T11:21:28.076641:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:21:33,457 nipype.interface INFO:
	 stderr 2021-11-16T11:21:33.457171:++ Setting up regressors
211116-11:21:33,458 nipype.interface INFO:
	 stderr 2021-11-16T11:21:33.457171:++ setting up stopband frequency mask
211116-11:21:33,458 nipype.interface INFO:
	 stderr 2021-11-16T11:21:33.457171: + Block #0: 550 time points -- 1 stopband regressors
211116-11:21:33,459 nipype.interface INFO:
	 stderr 2021-11-16T11:21:33.457171:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:21:33,460 nipype.interface INFO:
	 stderr 2021-11-16T11:21:33.457171: + -- 6 other fixed ort regressors
211116-11:21:33,460 nipype.interface INFO:
	 stderr 2021-11-16T11:21:33.457171:++ 550 retained time points MINUS 10 r

211116-11:22:51,504 nipype.interface INFO:
	 stderr 2021-11-16T11:22:51.503623:     Warnings of this type will be muted for this session.
211116-11:22:51,505 nipype.interface INFO:
	 stderr 2021-11-16T11:22:51.503623:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:22:55,457 nipype.interface INFO:
	 stderr 2021-11-16T11:22:55.457720:++ Setting up regressors
211116-11:22:55,458 nipype.interface INFO:
	 stderr 2021-11-16T11:22:55.457720:++ setting up stopband frequency mask
211116-11:22:55,458 nipype.interface INFO:
	 stderr 2021-11-16T11:22:55.457720: + Block #0: 550 time points -- 1 stopband regressors
211116-11:22:55,459 nipype.interface INFO:
	 stderr 2021-11-16T11:22:55.457720:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:22:55,459 nipype.interface INFO:
	 stderr 2021-11-16T11:22:55.457720: + -- 7 other fixed ort regressors
211116-11:22:55,460 nipype.interface INFO:
	 stderr 2021-11-16T11:22:55.457720:++ 550 retained time points MINUS 11

211116-11:24:04,77 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.077079:++ Setting up regressors
211116-11:24:04,77 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.077079:++ setting up stopband frequency mask
211116-11:24:04,78 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.077079: + Block #0: 550 time points -- 1 stopband regressors
211116-11:24:04,78 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.077079:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:24:04,79 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.077079: + -- 6 other fixed ort regressors
211116-11:24:04,79 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.077079:++ 550 retained time points MINUS 10 regressors ==> 540 D.O.F. left
211116-11:24:04,81 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.081086:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:24:04,81 nipype.interface INFO:
	 stderr 2021-11-16T11:24:04.081086:++ Compute pseudo-inverse of fixed orts
21

211116-11:25:14,578 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.578270:++ Setting up regressors
211116-11:25:14,578 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.578270:++ setting up stopband frequency mask
211116-11:25:14,579 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.578270: + Block #0: 550 time points -- 1 stopband regressors
211116-11:25:14,580 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.578270:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:25:14,580 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.578270: + -- 6 other fixed ort regressors
211116-11:25:14,581 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.578270:++ 550 retained time points MINUS 10 regressors ==> 540 D.O.F. left
211116-11:25:14,582 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.582061:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:25:14,582 nipype.interface INFO:
	 stderr 2021-11-16T11:25:14.582061:++ Compute pseudo-inverse of fixed

211116-11:26:22,117 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.116968:++ Setting up regressors
211116-11:26:22,117 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.116968:++ setting up stopband frequency mask
211116-11:26:22,118 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.116968: + Block #0: 550 time points -- 1 stopband regressors
211116-11:26:22,118 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.116968:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:26:22,119 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.116968: + -- 8 other fixed ort regressors
211116-11:26:22,120 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.116968:++ 550 retained time points MINUS 12 regressors ==> 538 D.O.F. left
211116-11:26:22,120 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.120895:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:26:22,121 nipype.interface INFO:
	 stderr 2021-11-16T11:26:22.120895:++ Compute pseudo-inverse of fixed

In [4]:
# find what percentage of scans are flagged per person
sub_motion_outliers = []
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses_num']+'/func/'+sub['subj_id']+'_'+sub['ses_num']+'_task-srmmovie_run-1_desc-confounds_regressors.tsv', sep='\t')
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    sub_motion_outliers.append(np.shape(confound_motion_outliers)[1]/np.shape(confound_motion_outliers)[0])

## loop through subjects and denoise barmaid

In [6]:
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    tproject.inputs.bandpass = (0.001, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run()  


211116-11:30:14,586 nipype.interface INFO:
	 stderr 2021-11-16T11:30:14.586686:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:30:14,588 nipype.interface INFO:
	 stderr 2021-11-16T11:30:14.586686:++ Authored by: Cox the Algebraic (Linear)
211116-11:30:14,605 nipype.interface INFO:
	 stderr 2021-11-16T11:30:14.605295:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-000/ses-01/func/sub-000_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:30:14,605 nipype.interface INFO:
	 stderr 2021-11-16T11:30:14.605295:     Warnings of this type will be muted for this session.
211116-11:30:14,606 nipype.interface INFO:
	 stderr 2021-11-16T11:30:14.605295:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:30:23,306 nipype.interface INFO:
	 stderr 2021-11-16T11:30:23.306949:++ Setting up regressors
211116-11:30:23,307 nipy

211116-11:32:02,494 nipype.interface INFO:
	 stderr 2021-11-16T11:32:02.494026:++ ===== clock time = 34s 746ms
211116-11:32:02,765 nipype.interface INFO:
	 stderr 2021-11-16T11:32:02.765396:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:32:02,767 nipype.interface INFO:
	 stderr 2021-11-16T11:32:02.765396:++ Authored by: Cox the Algebraic (Linear)
211116-11:32:02,780 nipype.interface INFO:
	 stderr 2021-11-16T11:32:02.780826:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-103/ses-01/func/sub-103_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:32:02,781 nipype.interface INFO:
	 stderr 2021-11-16T11:32:02.780826:     Warnings of this type will be muted for this session.
211116-11:32:02,782 nipype.interface INFO:
	 stderr 2021-11-16T11:32:02.780826:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:32:07,6

211116-11:33:47,531 nipype.interface INFO:
	 stderr 2021-11-16T11:33:47.530808:++ ===== clock time = 36s 545ms
211116-11:33:47,838 nipype.interface INFO:
	 stderr 2021-11-16T11:33:47.838259:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:33:47,840 nipype.interface INFO:
	 stderr 2021-11-16T11:33:47.838259:++ Authored by: Cox the Algebraic (Linear)
211116-11:33:47,841 nipype.interface INFO:
	 stderr 2021-11-16T11:33:47.841940:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-106/ses-01/func/sub-106_ses-01_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:33:47,842 nipype.interface INFO:
	 stderr 2021-11-16T11:33:47.841940:     Warnings of this type will be muted for this session.
211116-11:33:47,843 nipype.interface INFO:
	 stderr 2021-11-16T11:33:47.841940:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:33:50,8

211116-11:35:12,162 nipype.interface INFO:
	 stderr 2021-11-16T11:35:12.162595:++ Convert results to output dataset
211116-11:35:38,654 nipype.interface INFO:
	 stderr 2021-11-16T11:35:38.654081:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/confound_corrected_data/sub-109_ses-02_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz
211116-11:35:38,654 nipype.interface INFO:
	 stderr 2021-11-16T11:35:38.654081:++ ===== clock time = 36s 850ms
211116-11:35:38,903 nipype.interface INFO:
	 stderr 2021-11-16T11:35:38.903324:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:35:38,905 nipype.interface INFO:
	 stderr 2021-11-16T11:35:38.903324:++ Authored by: Cox the Algebraic (Linear)
211116-11:35:38,914 nipype.interface INFO:
	 stderr 2021-11-16T11:35:38.913991:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-111/ses-01/fun

211116-11:37:13,464 nipype.interface INFO:
	 stderr 2021-11-16T11:37:13.464481:*+ WARNING: 3dTproject input data :: 110633 vectors are constant
211116-11:37:13,490 nipype.interface INFO:
	 stderr 2021-11-16T11:37:13.490133:++ Starting project-orization
211116-11:37:14,806 nipype.interface INFO:
	 stderr 2021-11-16T11:37:14.806393:++ Convert results to output dataset
211116-11:37:41,710 nipype.interface INFO:
	 stderr 2021-11-16T11:37:41.710943:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/confound_corrected_data/sub-115_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz
211116-11:37:41,711 nipype.interface INFO:
	 stderr 2021-11-16T11:37:41.710943:++ ===== clock time = 46s 828ms
211116-11:37:41,967 nipype.interface INFO:
	 stderr 2021-11-16T11:37:41.966885:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:37:41,968 nipype.interface INFO:
	 stderr 2021-11-16T11:37:41.966885:++ Auth

211116-11:38:59,437 nipype.interface INFO:
	 stderr 2021-11-16T11:38:59.437330:++ Compute pseudo-inverse of fixed orts
211116-11:38:59,438 nipype.interface INFO:
	 stderr 2021-11-16T11:38:59.437330:++ Loading dataset
211116-11:38:59,439 nipype.interface INFO:
	 stderr 2021-11-16T11:38:59.437330:++ THD_dset_to_vectim: allocated 1093092000 bytes
211116-11:39:02,497 nipype.interface INFO:
	 stderr 2021-11-16T11:39:02.497542:*+ WARNING: 3dTproject input data :: 118749 vectors are constant
211116-11:39:02,552 nipype.interface INFO:
	 stderr 2021-11-16T11:39:02.552305:++ Starting project-orization
211116-11:39:03,430 nipype.interface INFO:
	 stderr 2021-11-16T11:39:03.430122:++ Convert results to output dataset
211116-11:39:29,594 nipype.interface INFO:
	 stderr 2021-11-16T11:39:29.594722:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/confound_corrected_data/sub-119_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.n

211116-11:40:44,98 nipype.interface INFO:
	 stderr 2021-11-16T11:40:44.095281:++ 840 retained time points MINUS 41 regressors ==> 799 D.O.F. left
211116-11:40:44,98 nipype.interface INFO:
	 stderr 2021-11-16T11:40:44.098527:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:40:44,99 nipype.interface INFO:
	 stderr 2021-11-16T11:40:44.098527:++ Compute pseudo-inverse of fixed orts
211116-11:40:44,101 nipype.interface INFO:
	 stderr 2021-11-16T11:40:44.101775:++ Loading dataset
211116-11:40:44,102 nipype.interface INFO:
	 stderr 2021-11-16T11:40:44.102763:++ THD_dset_to_vectim: allocated 1093092000 bytes
211116-11:40:47,203 nipype.interface INFO:
	 stderr 2021-11-16T11:40:47.203233:*+ WARNING: 3dTproject input data :: 110798 vectors are constant
211116-11:40:47,213 nipype.interface INFO:
	 stderr 2021-11-16T11:40:47.213910:++ Starting project-orization
211116-11:40:50,99 nipype.interface INFO:
	 stderr 2021-11-16T11:40:50.099410:++ Convert results to output dataset


211116-11:42:42,735 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.734149: + Block #0: 840 time points -- 3 stopband regressors
211116-11:42:42,736 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.734149:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:42:42,736 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.734149: + -- 6 other fixed ort regressors
211116-11:42:42,737 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.734149:++ 840 retained time points MINUS 12 regressors ==> 828 D.O.F. left
211116-11:42:42,738 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.738659:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:42:42,739 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.738659:++ Compute pseudo-inverse of fixed orts
211116-11:42:42,740 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.738659:++ Loading dataset
211116-11:42:42,740 nipype.interface INFO:
	 stderr 2021-11-16T11:42:42.738659:++ THD_dset_to_vectim: allocated 10930

211116-11:44:30,983 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.983854:++ Setting up regressors
211116-11:44:30,984 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.983854:++ setting up stopband frequency mask
211116-11:44:30,985 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.983854: + Block #0: 832 time points -- 3 stopband regressors
211116-11:44:30,986 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.983854:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:44:30,986 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.983854: + -- 48 other fixed ort regressors
211116-11:44:30,987 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.983854:++ 832 retained time points MINUS 54 regressors ==> 778 D.O.F. left
211116-11:44:30,988 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.988738:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:44:30,989 nipype.interface INFO:
	 stderr 2021-11-16T11:44:30.988738:++ Compute pseudo-inverse of fixe

In [7]:
# find what percentage of scans are flagged per person

sub_motion_outliers_barmaid = []
for i, sub in subjlist.iterrows():
#load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['barmaid_ses']+'/func/'+sub['subj_id']+'_'+sub['barmaid_ses']+'_task-movieviewing_'+ sub['barmaid_place']+ '_desc-confounds_regressors.tsv', sep='\t')
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    sub_motion_outliers_barmaid.append(np.shape(confound_motion_outliers)[1]/np.shape(confound_motion_outliers)[0])

## loop through subjects and denoise bluestreets

In [8]:
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    tproject.inputs.bandpass = (0.001, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run() 

211116-11:46:26,766 nipype.interface INFO:
	 stderr 2021-11-16T11:46:26.766286:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:46:26,768 nipype.interface INFO:
	 stderr 2021-11-16T11:46:26.766286:++ Authored by: Cox the Algebraic (Linear)
211116-11:46:26,781 nipype.interface INFO:
	 stderr 2021-11-16T11:46:26.781883:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-000/ses-01/func/sub-000_ses-01_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:46:26,784 nipype.interface INFO:
	 stderr 2021-11-16T11:46:26.781883:     Warnings of this type will be muted for this session.
211116-11:46:26,784 nipype.interface INFO:
	 stderr 2021-11-16T11:46:26.781883:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:46:33,755 nipype.interface INFO:
	 stderr 2021-11-16T11:46:33.755495:++ Setting up regressors
211116-11:46:33,756 nipy

211116-11:48:15,416 nipype.interface INFO:
	 stderr 2021-11-16T11:48:15.416063:++ ===== clock time = 35s 310ms
211116-11:48:15,678 nipype.interface INFO:
	 stderr 2021-11-16T11:48:15.678601:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:48:15,680 nipype.interface INFO:
	 stderr 2021-11-16T11:48:15.678601:++ Authored by: Cox the Algebraic (Linear)
211116-11:48:15,689 nipype.interface INFO:
	 stderr 2021-11-16T11:48:15.689211:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-103/ses-02/func/sub-103_ses-02_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:48:15,689 nipype.interface INFO:
	 stderr 2021-11-16T11:48:15.689211:     Warnings of this type will be muted for this session.
211116-11:48:15,691 nipype.interface INFO:
	 stderr 2021-11-16T11:48:15.689211:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:48:20,6

211116-11:50:02,357 nipype.interface INFO:
	 stderr 2021-11-16T11:50:02.356994:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/confound_corrected_data/sub-105_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz
211116-11:50:02,357 nipype.interface INFO:
	 stderr 2021-11-16T11:50:02.356994:++ ===== clock time = 34s 827ms
211116-11:50:02,747 nipype.interface INFO:
	 stderr 2021-11-16T11:50:02.747620:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:50:02,749 nipype.interface INFO:
	 stderr 2021-11-16T11:50:02.747620:++ Authored by: Cox the Algebraic (Linear)
211116-11:50:02,750 nipype.interface INFO:
	 stderr 2021-11-16T11:50:02.750935:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-106/ses-01/func/sub-106_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-11:50:

211116-11:51:22,66 nipype.interface INFO:
	 stderr 2021-11-16T11:51:22.066471:*+ WARNING: 3dTproject input data :: 115025 vectors are constant
211116-11:51:22,76 nipype.interface INFO:
	 stderr 2021-11-16T11:51:22.076544:++ Starting project-orization
211116-11:51:23,110 nipype.interface INFO:
	 stderr 2021-11-16T11:51:23.110447:++ Convert results to output dataset
211116-11:51:48,412 nipype.interface INFO:
	 stderr 2021-11-16T11:51:48.412265:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/confound_corrected_data/sub-109_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz
211116-11:51:48,413 nipype.interface INFO:
	 stderr 2021-11-16T11:51:48.412265:++ ===== clock time = 34s 840ms
211116-11:51:48,660 nipype.interface INFO:
	 stderr 2021-11-16T11:51:48.660579:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-11:51:48,662 nipype.interface INFO:
	 stderr 2021-11-16T11:51:48.660579:++ Author

211116-11:53:22,683 nipype.interface INFO:
	 stderr 2021-11-16T11:53:22.683112:++ Compute pseudo-inverse of fixed orts
211116-11:53:22,683 nipype.interface INFO:
	 stderr 2021-11-16T11:53:22.683112:++ Loading dataset
211116-11:53:22,684 nipype.interface INFO:
	 stderr 2021-11-16T11:53:22.683112:++ THD_dset_to_vectim: allocated 1061860800 bytes
211116-11:53:25,579 nipype.interface INFO:
	 stderr 2021-11-16T11:53:25.579427:*+ WARNING: 3dTproject input data :: 110405 vectors are constant
211116-11:53:25,625 nipype.interface INFO:
	 stderr 2021-11-16T11:53:25.625292:++ Starting project-orization
211116-11:53:26,775 nipype.interface INFO:
	 stderr 2021-11-16T11:53:26.774977:++ Convert results to output dataset
211116-11:53:52,649 nipype.interface INFO:
	 stderr 2021-11-16T11:53:52.649074:++ Output dataset /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/confound_corrected_data/sub-115_ses-02_task-movieviewing_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.n

211116-11:55:11,499 nipype.interface INFO:
	 stderr 2021-11-16T11:55:11.496996: + -- 6 other fixed ort regressors
211116-11:55:11,499 nipype.interface INFO:
	 stderr 2021-11-16T11:55:11.496996:++ 816 retained time points MINUS 12 regressors ==> 804 D.O.F. left
211116-11:55:11,500 nipype.interface INFO:
	 stderr 2021-11-16T11:55:11.500011:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:55:11,500 nipype.interface INFO:
	 stderr 2021-11-16T11:55:11.500011:++ Compute pseudo-inverse of fixed orts
211116-11:55:11,500 nipype.interface INFO:
	 stderr 2021-11-16T11:55:11.500011:++ Loading dataset
211116-11:55:11,502 nipype.interface INFO:
	 stderr 2021-11-16T11:55:11.500011:++ THD_dset_to_vectim: allocated 1061860800 bytes
211116-11:55:14,418 nipype.interface INFO:
	 stderr 2021-11-16T11:55:14.418864:*+ WARNING: 3dTproject input data :: 117338 vectors are constant
211116-11:55:14,465 nipype.interface INFO:
	 stderr 2021-11-16T11:55:14.465414:++ Starting project-orizatio

211116-11:56:55,148 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.148091:++ setting up stopband frequency mask
211116-11:56:55,149 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.148091: + Block #0: 816 time points -- 3 stopband regressors
211116-11:56:55,149 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.148091:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:56:55,150 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.148091: + -- 12 other fixed ort regressors
211116-11:56:55,151 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.148091:++ 816 retained time points MINUS 18 regressors ==> 798 D.O.F. left
211116-11:56:55,152 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.152510:++ no -mask option ==> processing all 325325 voxels in dataset
211116-11:56:55,153 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.152510:++ Compute pseudo-inverse of fixed orts
211116-11:56:55,153 nipype.interface INFO:
	 stderr 2021-11-16T11:56:55.152510:++ Loading dataset

211116-11:58:42,118 nipype.interface INFO:
	 stderr 2021-11-16T11:58:42.117341:     Warnings of this type will be muted for this session.
211116-11:58:42,120 nipype.interface INFO:
	 stderr 2021-11-16T11:58:42.117341:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-11:58:46,907 nipype.interface INFO:
	 stderr 2021-11-16T11:58:46.907934:++ Setting up regressors
211116-11:58:46,908 nipype.interface INFO:
	 stderr 2021-11-16T11:58:46.907934:++ setting up stopband frequency mask
211116-11:58:46,909 nipype.interface INFO:
	 stderr 2021-11-16T11:58:46.907934: + Block #0: 812 time points -- 3 stopband regressors
211116-11:58:46,909 nipype.interface INFO:
	 stderr 2021-11-16T11:58:46.907934:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-11:58:46,910 nipype.interface INFO:
	 stderr 2021-11-16T11:58:46.907934: + -- 6 other fixed ort regressors
211116-11:58:46,910 nipype.interface INFO:
	 stderr 2021-11-16T11:58:46.907934:++ 812 retained time points MINUS 12

211116-12:00:22,970 nipype.interface INFO:
	 stderr 2021-11-16T12:00:22.970016:     Warnings of this type will be muted for this session.
211116-12:00:22,971 nipype.interface INFO:
	 stderr 2021-11-16T12:00:22.970016:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-12:00:31,302 nipype.interface INFO:
	 stderr 2021-11-16T12:00:31.302942:++ Setting up regressors
211116-12:00:31,303 nipype.interface INFO:
	 stderr 2021-11-16T12:00:31.302942:++ setting up stopband frequency mask
211116-12:00:31,304 nipype.interface INFO:
	 stderr 2021-11-16T12:00:31.302942: + Block #0: 811 time points -- 4 stopband regressors
211116-12:00:31,304 nipype.interface INFO:
	 stderr 2021-11-16T12:00:31.302942:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-12:00:31,305 nipype.interface INFO:
	 stderr 2021-11-16T12:00:31.302942: + -- 14 other fixed ort regressors
211116-12:00:31,305 nipype.interface INFO:
	 stderr 2021-11-16T12:00:31.302942:++ 811 retained time points MINUS 2

In [9]:
# find what percentage of scans are flagged per person

sub_motion_outliers_bluestreets = []
for i, sub in subjlist.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['bluestreets_ses']+'/func/'+sub['subj_id']+'_'+sub['bluestreets_ses']+'_task-movieviewing_'+ sub['bluestreets_place']+ '_desc-confounds_regressors.tsv', sep='\t')
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    sub_motion_outliers_bluestreets.append(np.shape(confound_motion_outliers)[1]/np.shape(confound_motion_outliers)[0])

## loop through subjects and denoise recall

In [11]:
subj_recall=pd.read_excel(analysis_dir+'subj_recall_sessions_run.xlsx')

In [13]:
temp_dir = '/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/stats/'

In [14]:
for i, sub in subj_recall.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(temp_dir+'/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors_select.csv', header=False, index= False)
    

In [15]:
for i, sub in subj_recall.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['csf','white_matter']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion],axis = 1)
    confound_select.to_csv(temp_dir+'/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors_wm-csf.csv', header=False, index= False)
    

In [16]:
for i, sub in subj_recall.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors.tsv', sep='\t')
    index = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    confound_motion = confound_full[index]
    confound_select = pd.concat([confound_motion_outliers,confound_motion],axis = 1)
    confound_select.to_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors_select.1D', header=False, index= False)
    
    #perform filtering and denoising
    tproject = afni.TProject()
    tproject.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    #tproject.inputs.bandpass = (0.005, 99999)
    tproject.inputs.bandpass = (0.001, 99999)
    tproject.inputs.ort = (data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors_select.1D')
    tproject.inputs.automask = False
    tproject.inputs.out_file = analysis_dir+'confound_corrected_data/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_denoised_lenient_hp_filter.nii.gz'
    tproject.inputs.norm = False
    res = tproject.run() 

211116-13:10:01,228 nipype.interface INFO:
	 stderr 2021-11-16T13:10:01.228575:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-13:10:01,230 nipype.interface INFO:
	 stderr 2021-11-16T13:10:01.228575:++ Authored by: Cox the Algebraic (Linear)
211116-13:10:01,308 nipype.interface INFO:
	 stderr 2021-11-16T13:10:01.308487:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-000/ses-01/func/sub-000_ses-01_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-13:10:01,309 nipype.interface INFO:
	 stderr 2021-11-16T13:10:01.308487:     Warnings of this type will be muted for this session.
211116-13:10:01,309 nipype.interface INFO:
	 stderr 2021-11-16T13:10:01.308487:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-13:10:07,537 nipype.interface INFO:
	 stderr 2021-11-16T13:10:07.537947:++ Setting up regressors
211116-13:10:07,538 nipy

211116-13:12:59,488 nipype.interface INFO:
	 stderr 2021-11-16T13:12:59.487485:++ ===== clock time = 25s 228ms
211116-13:12:59,647 nipype.interface INFO:
	 stderr 2021-11-16T13:12:59.647681:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-13:12:59,649 nipype.interface INFO:
	 stderr 2021-11-16T13:12:59.647681:++ Authored by: Cox the Algebraic (Linear)
211116-13:12:59,658 nipype.interface INFO:
	 stderr 2021-11-16T13:12:59.658659:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-102/ses-02/func/sub-102_ses-02_task-spokenrecall_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-13:12:59,659 nipype.interface INFO:
	 stderr 2021-11-16T13:12:59.658659:     Warnings of this type will be muted for this session.
211116-13:12:59,660 nipype.interface INFO:
	 stderr 2021-11-16T13:12:59.658659:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-13:13:01,9

211116-13:13:51,736 nipype.interface INFO:
	 stderr 2021-11-16T13:13:51.736232:++ 3dTproject: AFNI version=AFNI_19.2.02 (Jul 17 2019) [64-bit]
211116-13:13:51,738 nipype.interface INFO:
	 stderr 2021-11-16T13:13:51.736232:++ Authored by: Cox the Algebraic (Linear)
211116-13:13:51,880 nipype.interface INFO:
	 stderr 2021-11-16T13:13:51.880753:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-105/ses-02/func/sub-105_ses-02_task-spokenrecall_run-2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-13:13:51,881 nipype.interface INFO:
	 stderr 2021-11-16T13:13:51.880753:     Warnings of this type will be muted for this session.
211116-13:13:51,882 nipype.interface INFO:
	 stderr 2021-11-16T13:13:51.880753:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-13:14:00,321 nipype.interface INFO:
	 stderr 2021-11-16T13:14:00.321362:++ Setting up regressors
211116-13:14:00,321 nipy

211116-13:15:07,608 nipype.interface INFO:
	 stderr 2021-11-16T13:15:07.608505:** AFNI converts NIFTI_datatype=4 (INT16) in file /Users/alexbarnett/Documents/TTTvsPMAT/fMRI/fmriprep_v20.1.1/fmriprep/sub-108/ses-02/func/sub-108_ses-02_task-spokenrecall_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz to FLOAT32
211116-13:15:07,609 nipype.interface INFO:
	 stderr 2021-11-16T13:15:07.608505:     Warnings of this type will be muted for this session.
211116-13:15:07,609 nipype.interface INFO:
	 stderr 2021-11-16T13:15:07.608505:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-13:15:10,407 nipype.interface INFO:
	 stderr 2021-11-16T13:15:10.407512:++ Setting up regressors
211116-13:15:10,408 nipype.interface INFO:
	 stderr 2021-11-16T13:15:10.407512:++ setting up stopband frequency mask
211116-13:15:10,408 nipype.interface INFO:
	 stderr 2021-11-16T13:15:10.407512: + Block #0: 270 time points -- 1 stopband regressors
211116-13:15:10,409 nipype.interface IN

211116-13:15:54,142 nipype.interface INFO:
	 stderr 2021-11-16T13:15:54.142285:     Warnings of this type will be muted for this session.
211116-13:15:54,143 nipype.interface INFO:
	 stderr 2021-11-16T13:15:54.142285:     Set AFNI_NIFTI_TYPE_WARN to YES to see them all, NO to see none.
211116-13:15:56,505 nipype.interface INFO:
	 stderr 2021-11-16T13:15:56.505582:++ Setting up regressors
211116-13:15:56,506 nipype.interface INFO:
	 stderr 2021-11-16T13:15:56.505582:++ setting up stopband frequency mask
211116-13:15:56,506 nipype.interface INFO:
	 stderr 2021-11-16T13:15:56.505582: + Block #0: 255 time points -- 2 stopband regressors
211116-13:15:56,507 nipype.interface INFO:
	 stderr 2021-11-16T13:15:56.505582:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-13:15:56,507 nipype.interface INFO:
	 stderr 2021-11-16T13:15:56.505582: + -- 15 other fixed ort regressors
211116-13:15:56,508 nipype.interface INFO:
	 stderr 2021-11-16T13:15:56.505582:++ 255 retained time points MINUS 2

...

211116-13:25:14,879 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.879935:++ Setting up regressors
211116-13:25:14,880 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.879935:++ setting up stopband frequency mask
211116-13:25:14,880 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.879935: + Block #0: 515 time points -- 2 stopband regressors
211116-13:25:14,881 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.879935:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-13:25:14,881 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.879935: + -- 94 other fixed ort regressors
211116-13:25:14,882 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.879935:++ 515 retained time points MINUS 99 regressors ==> 416 D.O.F. left
211116-13:25:14,883 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.883561:++ no -mask option ==> processing all 325325 voxels in dataset
211116-13:25:14,884 nipype.interface INFO:
	 stderr 2021-11-16T13:25:14.883561:++ Compute pseudo-inverse of fixe

211116-13:26:10,697 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.697758:++ Setting up regressors
211116-13:26:10,698 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.697758:++ setting up stopband frequency mask
211116-13:26:10,699 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.697758: + Block #0: 281 time points -- 2 stopband regressors
211116-13:26:10,699 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.697758:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-13:26:10,700 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.697758: + -- 10 other fixed ort regressors
211116-13:26:10,700 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.697758:++ 281 retained time points MINUS 15 regressors ==> 266 D.O.F. left
211116-13:26:10,701 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.701174:++ no -mask option ==> processing all 325325 voxels in dataset
211116-13:26:10,701 nipype.interface INFO:
	 stderr 2021-11-16T13:26:10.701174:++ Compute pseudo-inverse of fixe

211116-13:27:07,160 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.159567:++ setting up stopband frequency mask
211116-13:27:07,160 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.159567: + Block #0: 168 time points -- 1 stopband regressors
211116-13:27:07,161 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.159567:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-13:27:07,162 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.159567: + -- 19 other fixed ort regressors
211116-13:27:07,162 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.159567:++ 168 retained time points MINUS 23 regressors ==> 145 D.O.F. left
211116-13:27:07,164 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.163981:++ no -mask option ==> processing all 325325 voxels in dataset
211116-13:27:07,164 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.163981:++ Compute pseudo-inverse of fixed orts
211116-13:27:07,164 nipype.interface INFO:
	 stderr 2021-11-16T13:27:07.163981:++ Loading dataset

211116-13:28:00,633 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.632865:++ setting up stopband frequency mask
211116-13:28:00,633 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.632865: + Block #0: 456 time points -- 1 stopband regressors
211116-13:28:00,634 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.632865:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-13:28:00,634 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.632865: + -- 6 other fixed ort regressors
211116-13:28:00,635 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.632865:++ 456 retained time points MINUS 10 regressors ==> 446 D.O.F. left
211116-13:28:00,635 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.635861:++ no -mask option ==> processing all 325325 voxels in dataset
211116-13:28:00,637 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.635861:++ Compute pseudo-inverse of fixed orts
211116-13:28:00,637 nipype.interface INFO:
	 stderr 2021-11-16T13:28:00.635861:++ Loading dataset


211116-13:29:31,465 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.465849:++ Setting up regressors
211116-13:29:31,466 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.465849:++ setting up stopband frequency mask
211116-13:29:31,466 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.465849: + Block #0: 324 time points -- 1 stopband regressors
211116-13:29:31,467 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.465849:++ 1 Blocks * 3 polynomials -- 3 polort regressors
211116-13:29:31,468 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.465849: + -- 24 other fixed ort regressors
211116-13:29:31,469 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.465849:++ 324 retained time points MINUS 28 regressors ==> 296 D.O.F. left
211116-13:29:31,469 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.469569:++ no -mask option ==> processing all 325325 voxels in dataset
211116-13:29:31,470 nipype.interface INFO:
	 stderr 2021-11-16T13:29:31.469569:++ Compute pseudo-inverse of fixe

In [17]:
sub_motion_outliers_recall =[]
for i, sub in subj_recall.iterrows():
    #load in confounds and select relevant columns
    confound_full = pd.read_csv(data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+ '_desc-confounds_regressors.tsv', sep='\t')
    confound_motion_outliers = confound_full.filter(regex="motion_outlier")
    sub_motion_outliers_recall.append(np.shape(confound_motion_outliers)[1]/np.shape(confound_motion_outliers)[0])

In [20]:
analysis_dir

'/Users/alexbarnett/Documents/TTTvsPMAT/fMRI/SRM_2020/'

In [21]:
np.save(analysis_dir+'sub_motion_outliers_recall.npy',sub_motion_outliers_recall)

## calculate tSNR

In [32]:
from nipype.algorithms.confounds import TSNR

In [26]:
tsnr = TSNR()
tsnr.inputs.in_file = data_dir+'sub-000/ses-01/func/sub-000_ses-01_task-movieviewing_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii'
res = tsnr.run()

In [44]:
for i, sub in subj_recall.iterrows():
    tsnr = TSNR()
    tsnr.inputs.in_file = data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
    tsnr.inputs.tsnr_file = data_dir+sub['subj_id']+'/'+sub['ses']+'/func/'+sub['subj_id']+'_'+sub['ses']+'_task-spokenrecall_'+ sub['run']+'_space-MNI152NLin2009cAsym_desc-preproc_bold_tsnr.nii.gz'
    res = tsnr.run() 